In [1]:
from core.multi_case_modeller import MultiCaseModeller
from sixgill.definitions import ModelComponents, Parameters
from sixgill.pipesim import Model, Units
from core import ExcelInputError
from project import (
    get_string_values_from_class,
    generate_dict_from_class,
    get_class_by_name,
)
import pandas as pd
from core.network_simulation import NetworkSimulator

In [2]:
m = r"F:\Project\AET01083.00 Wave-Component_3A\09. Documents, Drawings\9.31 Process\09.31.01 Current documents\BAB Pump Hydraulics\1. HP Model\Rev 2\Models\Overall-Max_W-LP-EO_BAB HP.pips"

In [ ]:
ns = NetworkSimulator(m)
ns.run_existing_model()